In [1]:
import numpy as np 
import h5py 
import sys,os
os.chdir('../../')
sys.path.append('./src/data')
sys.path.append('./src/events')
sys.path.append('./src/probability')
import pandas as pd 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
from processerDC import get_flux, get_interpolators_dc
interp_flux, interp_aeff = get_interpolators_dc()

In [2]:
Ereco = np.array([5.623413,  7.498942, 10. , 13.335215, 17.782795, 23.713737, 31.622776, 42.16965 , 56.23413])
zreco = np.array([-1., -0.75, -0.5 , -0.25,  0., 0.25, 0.5, 0.75, 1.])

In [34]:
filename = 'C:/Users/Martin/Documents/IceCubeUpgradeNeutrinoMCDataRelease/events/neutrino_mc.csv'
df = (pd.read_csv(filename))
        #.query('pid == 1') #only tracks
        #.query('pdg==14 or pdg==-14')) #only muon (anti)neutrinos
df.reco_zenith = np.cos(df.reco_zenith)
df.true_zenith = np.cos(df.true_zenith)
#df['Ebin'] = pd.cut(df.reco_energy, bins=Ereco, labels=False)
#df['zbin'] = pd.cut(df.reco_zenith, bins=zreco, labels=False)
#df = df.dropna()

nue_mask = (np.abs(df["pdg"]) == 12)
numu_mask = (np.abs(df["pdg"]) == 14)
nutau_mask = (np.abs(df["pdg"]) == 16)
nc_mask = df["current_type"] == 0
cc_mask = df["current_type"] == 1
nue_cc_mask = nue_mask & cc_mask
numu_cc_mask = numu_mask & cc_mask
nutau_cc_mask = nutau_mask & cc_mask

In [35]:
df.query('zbin<=3')

UndefinedVariableError: name 'zbin' is not defined

In [37]:
numu_flux = get_flux('m',df['true_energy'],df['true_zenith'],interp_flux)
nue_flux = get_flux('e',df['true_energy'],df['true_zenith'],interp_flux)

In [46]:
Pmutau = 0.3*np.ones_like(df["weight"])

In [47]:
rate_weight = np.zeros_like(df["weight"])
rate_weight[nue_mask] = nue_flux[nue_mask] * df["weight"][nue_mask] # No oscillations in 2 flavor model
rate_weight[numu_mask] = numu_flux[numu_mask] * (1. - Pmutau[numu_mask]) * df["weight"][numu_mask] # numu flux, less those that oscillation
rate_weight[nutau_mask] = numu_flux[nutau_mask] * Pmutau[nutau_mask] * df["weight"][nutau_mask] # nutau purely come from oscillations in the numu flux
df["rate_weight"] = rate_weight

In [48]:
df

,pdg,pid,true_energy,true_zenith,true_azimuth,reco_energy,reco_zenith,reco_azimuth,interaction_type,current_type,weight,xsec,dxsec,x,y,W,Q2,rate_weight
0,12,0,3.46,-0.580387,1.07,4.97,-0.658786,1.45,1,1,8.980000e-09,0.782,5.68,0.0879,0.1130,1.250,0.0648,2.106473e-08
1,12,0,12.30,0.020795,5.39,13.20,0.120503,5.86,2,1,2.180000e-07,70.200,116.00,0.4470,0.5520,2.820,5.7100,2.809675e-08
2,12,0,12.30,-0.069148,1.13,14.60,0.070737,1.23,2,1,2.170000e-07,69.900,138.00,0.1020,0.7410,4.030,1.7400,2.640792e-08
3,12,0,4.15,-0.069148,5.62,6.37,-0.079121,5.62,1,1,1.290000e-08,6.260,44.50,0.3510,0.1140,1.210,0.3130,3.676744e-08
4,12,0,1.89,-0.563985,3.33,2.88,-0.478727,3.16,0,1,1.450000e-09,7.600,1.11,2.0600,0.0374,0.861,0.2730,1.987210e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271778,-16,0,47.60,-0.323290,2.99,20.10,0.659232,3.01,2,0,9.600000e-06,1.080,6.61,0.1150,0.1130,3.130,1.1600,9.814151e-09
271779,-16,1,51.00,-0.049184,1.91,31.50,0.305817,1.70,2,0,1.150000e-05,4.790,22.80,0.0833,0.6940,7.870,5.5500,1.366070e-08
271780,-16,0,124.00,0.020795,2.61,37.50,-0.275590,2.69,2,0,1.170000e-04,50.000,86.30,0.3360,0.2990,6.870,23.4000,1.027606e-08
271781,-16,0,16.70,0.948287,2.23,12.70,0.647117,2.60,2,0,6.590000e-07,2.920,1.49,0.2230,0.7280,4.310,5.0900,1.089874e-08


In [49]:
np.sum(df.rate_weight)*1e3

2.534646902667058